In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[:logpop] = map((x) -> log(x), df[:population]);
df[:society] = 1:10;

first(df[[:total_tools, :logpop, :society]], 5)

struct m_12_06d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_12_06d_model)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, σ = θ  # β : a, bp, α : a_society
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ)
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += logpdf.(Normal(0, 10), β[1]) # a
    ll += logpdf.(Normal(0, 1), β[2]) # a
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
N_societies = length(unique(df[:society]))
X = hcat(ones(Int64, N), df[:logpop]);
S = df[:society]
y = df[:total_tools]
p = m_12_06d_model(y, X, S, N, N_societies);
θ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), σ = 0.2)
p(θ)

-242.9022282091119

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_12_06d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_societies), σ = asℝ₊) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.003 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0035 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0036 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0016 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00074 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00048 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00033 s/step ...done
MCMC (1000 steps)
0.00034 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α, :σ),Tuple{Array{Float64,1},Array{Float64,1},Float64}},1}:
 (β = [2.732417138647978, 0.1127490826639533], α = [-0.9367872237607355, -0.5201117375488722, -0.46666982080558367, 0.07648925802348758, -0.14389840330736264, -0.8172026168457225, 0.17841456274554485, -0.2725581225884127, 0.26298966005737306, 0.16333220705568352], σ = 0.5065192357989874)  
 (β = [0.5234399371848654, 0.3267982523628996], α = [0.052914837148009305, -0.05682547224113979, 0.0949255985234548, 0.5883967890719909, -0.133057664501214, -0.3699365944386078, -0.1570899288416014, -0.4851449508506881, 0.1647595287150635, -0.533226086334198], σ = 0.3197575834036804)      
 (β = [0.9668928768259561, 0.2625342945299601], α = [-0.15374519631646294, 0.20414096748211935, 0.20256305111974665, 0.7247815902657849, -0.06733052427826251, -0.564909356422915, 0.20555577953420312, -0.08291901529938603, 0.3335657533604276, -0.24531930893939183], σ = 0.3901342077737331)  
 (β = [0.7364029106623591, 0.2895

Extract the parameter posterior means.

In [8]:
posterior_β = mean(posterior[i].β for i in 1:length(posterior))
posterior_α = mean(posterior[i].α for i in 1:length(posterior))
posterior_σ = mean(posterior[i].σ for i in 1:length(posterior))

0.3034384449616687

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 872.461  846.517  963.326  1000.0  …  877.596  482.21  724.03  58.3527

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.87, min/25%/median/75%/max: 0.03 0.84 0.9 0.93 0.97
  termination: AdjacentTurn => 4% DoubledTurn => 96%
  depth: 2 => 0% 3 => 7% 4 => 93% 5 => 0%


CmdStan result

In [11]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [12]:
[posterior_β, posterior_α, [posterior_σ]]

3-element Array{Array{Float64,1},1}:
 [1.1104795953484992, 0.25995221045980044]                                                                                                                                                                          
 [-0.19534360289041014, 0.041083362708320845, -0.04389073543031416, 0.3227268545473028, 0.04329288204903583, -0.3055729975073223, 0.1433460949620147, -0.16710067703462808, 0.275567469186962, -0.09053669260892704]
 [0.3034384449616687]                                                                                                                                                                                               

End of m12.6d.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*